Tensor Flow Data Validation: https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic

Tensor Flow Extension: https://www.tensorflow.org/tfx

Census Income Data Set: http://archive.ics.uci.edu/ml/datasets/Census+Income

In [ ]:
#!pip install tensorflow_data_validation

In [2]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

from tensorflow_metadata.proto.v0 import schema_pb2

print('TFDV Version: {}'.format(tfdv.__version__))
print('Tensorflow Version: {}'.format(tf.__version__))

TFDV Version: 1.14.0
Tensorflow Version: 2.15.0


In [4]:
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', skipinitialspace=True, header=None)

In [5]:
train_df, eval_df = train_test_split(df, test_size=0.2, shuffle=False)

In [ ]:
train_df.head()

In [ ]:
eval_df.head()

In [12]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6513 entries, 26048 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6513 non-null   int64 
 1   1       6513 non-null   object
 2   2       6513 non-null   int64 
 3   3       6513 non-null   object
 4   4       6513 non-null   int64 
 5   5       6513 non-null   object
 6   6       6513 non-null   object
 7   7       6513 non-null   object
 8   8       6513 non-null   object
 9   9       6513 non-null   object
 10  10      6513 non-null   int64 
 11  11      6513 non-null   int64 
 12  12      6513 non-null   int64 
 13  13      6513 non-null   object
 14  14      6513 non-null   object
dtypes: int64(6), object(9)
memory usage: 814.1+ KB


In [10]:
novos_dados = [[46,'NaN',257473,'Bachelors',8,'Married-civ-spouse','Plumber','Husband','Other','Male',1000,0,41,'Australia','>50K'],
               [0,'Private',257473,'Masters',8,'Married-civ-spouse','Adm-clerical','Wife','Asian','Female',0,0,40,'Pakistan','>50K'],
               [1000,'Private',257473,'Masters',8,'Married-civ-spouse','Prof-specialty','Husband','Black','Male',0,0,20,'Cameroon','<=50K'],
               [25,'?',257473,'Masters',8,'Married-civ-spouse','gamer','Husband','Asian','Female',0,0,50,'Mongolia','<=50K']]

eval_df2 = eval_df.append(novos_dados, ignore_index=True)


<ipython-input-10-c1f101362be6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df2 = eval_df.append(novos_dados, ignore_index=True)


In [11]:
eval_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6517 entries, 0 to 6516
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6517 non-null   int64 
 1   1       6517 non-null   object
 2   2       6517 non-null   int64 
 3   3       6517 non-null   object
 4   4       6517 non-null   int64 
 5   5       6517 non-null   object
 6   6       6517 non-null   object
 7   7       6517 non-null   object
 8   8       6517 non-null   object
 9   9       6517 non-null   object
 10  10      6517 non-null   int64 
 11  11      6517 non-null   int64 
 12  12      6517 non-null   int64 
 13  13      6517 non-null   object
 14  14      6517 non-null   object
dtypes: int64(6), object(9)
memory usage: 763.8+ KB


In [13]:
eval_df2.tail(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
6512,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K
6513,46,NaN,257473,Bachelors,8,Married-civ-spouse,Plumber,Husband,Other,Male,1000,0,41,Australia,>50K
6514,0,Private,257473,Masters,8,Married-civ-spouse,Adm-clerical,Wife,Asian,Female,0,0,40,Pakistan,>50K
6515,1000,Private,257473,Masters,8,Married-civ-spouse,Prof-specialty,Husband,Black,Male,0,0,20,Cameroon,<=50K
6516,25,?,257473,Masters,8,Married-civ-spouse,gamer,Husband,Asian,Female,0,0,50,Mongolia,<=50K


## Geração e visualização do dataset de treino

In [14]:
estatistica_treino = tfdv.generate_statistics_from_dataframe(train_df)

In [15]:
tfdv.visualize_statistics(estatistica_treino)

In [16]:
schema = tfdv.infer_schema(statistics=estatistica_treino)

tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'0',INT,required,,-
'1',STRING,required,,'1'
'2',INT,required,,-
'3',STRING,required,,'3'
'4',INT,required,,-
'5',STRING,required,,'5'
'6',STRING,required,,'6'
'7',STRING,required,,'7'
'8',STRING,required,,'8'


,Values
Domain,
'1',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"
'3',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'5',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'6',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'7',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'8',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'9',"'Female', 'Male'"
'13',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'14',"'<=50K', '>50K'"


## Geração e visualização do dataset de score

In [17]:
estatistica_score = tfdv.generate_statistics_from_dataframe(eval_df2)

tfdv.visualize_statistics(
    lhs_statistics=estatistica_score,
    rhs_statistics=estatistica_treino,
    lhs_name='SCORE',
    rhs_name='TREINO'
)

In [18]:
eval_df2 = eval_df2[eval_df2[0] > 16]
eval_df2 = eval_df2[eval_df2[0] < 91]

eval_df2.dropna(inplace=True)

In [19]:
estatistica_score2 = tfdv.generate_statistics_from_dataframe(eval_df2)

tfdv.visualize_statistics(
    lhs_statistics=estatistica_score2,
    rhs_statistics=estatistica_treino,
    lhs_name='SCORE',
    rhs_name='TREINO'
)

## Encontrar anomalias

In [20]:
anomalia =  tfdv.validate_statistics(statistics=estatistica_score2, schema=schema)

tfdv.display_anomalies(anomalia)

,Anomaly short description,Anomaly long description
Feature name,,
'6',Unexpected string values,"Examples contain values missing from the schema: Plumber (<1%), gamer (<1%)."
'1',Unexpected string values,Examples contain values missing from the schema: NaN (<1%).
'13',Unexpected string values,"Examples contain values missing from the schema: Australia (<1%), Mongolia (<1%)."
'8',Unexpected string values,Examples contain values missing from the schema: Asian (<1%).


In [21]:
country_feature = tfdv.get_feature(schema, '13')
country_feature.distribution_constraints.min_domain_mass = 0.9

occupation_feature = tfdv.get_feature(schema, '6')
occupation_feature.distribution_constraints.min_domain_mass = 0.9

In [22]:
race_domain = tfdv.get_domain(schema, '8')
race_domain.value.append('Asian')

In [23]:
tfdv.set_domain(schema, '0', schema_pb2.IntDomain(name='0', min=17, max=90))
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'0',INT,required,,min: 17; max: 90
'1',STRING,required,,'1'
'2',INT,required,,-
'3',STRING,required,,'3'
'4',INT,required,,-
'5',STRING,required,,'5'
'6',STRING,required,,'6'
'7',STRING,required,,'7'
'8',STRING,required,,'8'


,Values
Domain,
'1',"'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"
'3',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'5',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'6',"'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'7',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'8',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White', 'Asian'"
'9',"'Female', 'Male'"
'13',"'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
'14',"'<=50K', '>50K'"


In [24]:
anomalias_atualizadas = tfdv.validate_statistics(estatistica_score2, schema)


In [25]:
tfdv.display_anomalies(anomalias_atualizadas)

,Anomaly short description,Anomaly long description
Feature name,,
'1',Unexpected string values,Examples contain values missing from the schema: NaN (<1%).


In [26]:
workclass_domain = tfdv.get_domain(schema, '1')
workclass_domain.value.append('NaN')

In [27]:
anomalias_atualizadas = tfdv.validate_statistics(estatistica_score2, schema)

In [28]:
tfdv.display_anomalies(anomalias_atualizadas)